In [1]:
from Pipeline import Helper, Purchase_Probabilities
import pandas as pd

In [106]:
DATA_PATH   =   r'C:/Users/Benedikt/Documents/GitHub/mlim-g2/data/'

EXPORT_PATH =   r'C:/Users/Benedikt/Documents/GitHub/mlim-g2/data/export'

files = {
    'data': [
        #DATA_PATH + 'baskets.parquet',
        #DATA_PATH + 'coupon_index.parquet', 
        #DATA_PATH + 'coupons.parquet', 
        DATA_PATH + 'merged.parquet',
        DATA_PATH + 'clean.parquet', 
        #DATA_PATH + 'prepare.parquet'
    ]}

In [136]:
basket = pd.read_parquet( r'C:/Users/Benedikt/Documents/GitHub/mlim-g2/data/clean.parquet')
basket

,week,shopper,product,price,discount,purchased
0,0,0,71,629.0,0.00,1
1,0,0,91,605.0,0.00,1
2,0,0,116,715.0,0.00,1
3,0,0,123,483.0,0.00,1
4,0,0,157,592.0,0.00,1
...,...,...,...,...,...,...
110673244,89,1998,101,NaN,0.25,0
110673245,89,1999,207,NaN,0.25,0
110673246,89,1999,246,NaN,0.30,0
110673247,89,1999,51,NaN,0.15,0


In [137]:
class Elasticities(Helper):
    
    """calculate price elasticities
    
        input: week(touple: start, stop), basket_df, coupons_df
        
        output: elasticity_df 
    """
    
    def __init__(self):
        super().__init__() 
        self.basket = basket
        self.coupons = coupons    
    
    def calc_week_elasticity(self, week):  
        # runtime of 3.3 sec
        
        basket = baskets
        
        basket_week = basket[(week[1] >= basket['week']) & (basket['week'] >= week[0])].copy()
        coupons_week = coupons[(week[1] >= coupons['week']) & (coupons['week'] >= week[0])].copy()
        basket_week = basket_week.merge(coupons_week, how = "outer")
        
        basket_week['discount'] = basket_week['discount'].fillna(0)
        basket_week['price'] = basket_week['price'].fillna(0)

        elast = []
        total_basket_count = 100000 * (week[1] - week[0] + 1)

        for i in range(250):
            reg_price = basket_week[basket_week['product'] == i]
            reg_price_buy = len(reg_price[reg_price['discount'] == 0])
            all_discounts_offers = len(reg_price[reg_price['discount'] > 0])
            reg_price_offer = total_basket_count - all_discounts_offers
            reg_price_buy_rate = reg_price_buy / reg_price_offer
            discount_30 = reg_price[reg_price['discount'] == 30]
            discount_30_offer = len(discount_30)
            discount_30_buy = (discount_30['price'] != 0).sum()
            discount_buy_rate = discount_30_buy / discount_30_offer
            elast.append((discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))

        elasticity_df = pd.DataFrame(elast)
        elasticity_df['product'] = elasticity_df.index 
        return elasticity_df
    
    
class Redemptions(Helper):
    
    
    """calculate product and costumer specific coupon redemption likelihoods
    
        input: 
        week (touple, start, stop) 
        shopper (touple, start, stop)
        basket df variables: 'discount'(int/float), 'price'(int/float), 'product'(int), 'shopper'(int) (nan set to zero)
        
        output: basket df with 'product_redemption_likelihood', 'shopper_product_redemption_likelihood', 'discount_buy' added
    
    """
    
    def __init__(self):
        super().__init__()
        
    def shopper_coupon_rates(self, basket, week, shopper):
        # data is shopper specific => use shoppers [0-1999] 
        # 0.1 sec for 1 week & 2000 shoppers    
        # data is very sparse if only one week is used 
            
        output = basket[(week[1] >= basket['week']) & (basket['week'] >= week[0])].copy()              # delete if specific weeks already as input
        output = output[(shopper[1] >= output['shopper']) & (output['shopper'] >= shopper[0])]         # delete if only 2000 shoppers as input
        coupons = output[output['discount'] > 0].copy()
        coupons.loc[(coupons['price'].notnull()), 'redeemed']  = 1
        coupons['redeemed'] = coupons['redeemed'].fillna(0)

        # coupon redemption likelihood by costumer & product
        costumer_redemption_rate = coupons.groupby(['shopper', 'product'])['redeemed'].mean()
        output = output.merge(costumer_redemption_rate, how = 'left', on = ['shopper', 'product'])
        output = output.rename(columns = {'redeemed' : 'shopper_product_redemption_likelihood'})


        # only bought because of discount?
        buy_all = output.groupby(['shopper', 'product']).size()
        discount = coupons.groupby(['shopper', 'product']).size()
        discount_buy = discount / buy_all
        discount_buy = discount_buy.fillna(0)
        discount_buy = discount_buy.rename('discount_buy')
        output = output.merge(discount_buy, how = 'left', on = ['shopper', 'product'])

        return output
    
    
    def product_coupon_rates(self, week, shopper):
        # is not completely shopper specific, data from 2000 shoppers is not accurate enough, min 10.000
        
        output = basket[(week[1] >= basket['week']) & (basket['week'] >= week[0])]
        output = output[(shopper[1] >= output['shopper']) & (output['shopper'] >= output[0])]

        # coupon redemption likelihood by product
        coupons = output[output['discount'] > 0].copy()
        coupons.loc[(coupons['price'] != 0), 'redeemed']  = 1
        coupons['redeemed'] = coupons['redeemed'].fillna(0)

        redemption_rate = coupons.groupby(['product'])['redeemed'].mean()
        output = output.merge(redemption_rate, how = 'left', on = 'product')
        output = output.rename(columns = {'redeemed' : 'product_redemption_likelihood'})
        
        return output

In [142]:
redemptions.shopper_coupon_rates(basket,  week , shopper)

,week,shopper,product,price,discount,purchased,shopper_product_redemption_likelihood,discount_buy
0,1,0,71,629.0,0.00,1,1.0,0.125
1,1,0,91,605.0,0.00,1,NaN,0.000
2,1,0,130,613.0,0.00,1,NaN,0.000
3,1,0,157,592.0,0.00,1,NaN,0.000
4,1,0,167,582.0,0.00,1,NaN,0.000
...,...,...,...,...,...,...,...,...
124134,10,1000,62,NaN,0.35,0,0.0,1.000
124135,10,1000,44,NaN,0.20,0,0.0,1.000
124136,10,1000,222,NaN,0.30,0,0.0,1.000
124137,10,1000,156,NaN,0.10,0,0.0,0.400


In [141]:
redemptions = Redemptions()

In [140]:
week = (1, 10)
shopper = (0, 1000)